In [1]:
import re
import os
import requests
import pytesseract
import numpy as np
import pandas as pd
import ipywidgets as ipw
from docx import Document
from datetime import datetime
from docxtpl import DocxTemplate
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from pdf2image import convert_from_bytes
from python_docx_replace import docx_blocks
os.environ['TESSDATA_PREFIX'] = r'tessdata'

directorio_credenciales = 'credentials_module.json'
def login():
    gauth = GoogleAuth()
    gauth.LoadCredentialsFile(directorio_credenciales)
    
    if gauth.access_token_expired:
        gauth.Refresh()
        gauth.SaveCredentialsFile(directorio_credenciales)
    else:
        gauth.Authorize()
        
    return GoogleDrive(gauth)
drive = login()

data_sgu = pd.read_excel('https://drive.google.com/uc?id=1lsIrNeWh4ol21MUzVAaI0k2ZPZm4WsPt').fillna('Completar')
data_rur = pd.read_excel('https://drive.google.com/uc?id=1MtNeZ_89MJJZ_5gQn67szh2I74hvGskO').fillna('Completar')

In [5]:
nomruc = ['Electro Dunas S.A.A.', 'Luz del Sur S.A.A.', 'Electro Ucayali S.A.', 'Electro Puno S.A.A.',
         'Empresa de Administración de Infraestructura Eléctrica S.A. - Adinelsa', 'Hidrandina S.A.','Sociedad Eléctrica del Sur Oeste S.A.', 'Contugas S.A.', 'Empresa de Servicios Electricos Municipales de Paramonga S.A. - Emsem S.A.', 'Electronorte S.A.',
         'Electrosur S.A.', 'Gases del Pacífico S.A.C. – Quavii', 'Electronoroeste S.A.', 'Electro Oriente S.A.',
         'Electrocentro S.A.', 'Empresa de Interes Local Hidroelectrica S.A. de Chacas', 'Enel Distribución Perú S.A.A.', 'Consorcio Electrico de Villacuri S.A.C',
         'Gas Natural de Lima y Callao S.A. – Cálidda', 'Electro Sur Este S.A.A.']
conces = ['ELECTRO DUNAS S.A.A.', 'LUZ DEL SUR S.A.A.', 'ELECTRO UCAYALI S.A.', '*ELECTRO PUNO S.A.A.',
         'ADINELSA', 'HIDRANDINA S.A.A.', 'SEAL S.A.', 'CONTUGAS S.A.C', 'EMSEMSA', 'ELECTRO NORTE S.A.',
         'ELECTRO SUR S.A.', 'GASES DEL PACIFICO S.A.C.', 'ELECTRO NOROESTE S.A.', 'ELECTRO ORIENTE S.A.',
         'ELECTRO CENTRO S.A.', 'EILHICHA S.A.', 'Enel Distribucion Peru SAA', 'COELVISA',
         'CALIDDA - GAS NATURAL DE LIMA Y CALLAO S.R.L.', 'ELECTRO SUR ESTE S.A.']
concer = ['EDN', 'ENO', 'EUC', 'ELN', 'QUAVVI',
          'SEA', 'ELC', 'ELS', 'ESM', 'ESE',
          'EDS', 'EOR', 'HID', 'CAL', '',
          '', '', '', '', '',]
ru2sgu = ['Enel Distribucion Peru SAA', 'ELECTRO NOROESTE S.A.', 'ELECTRO UCAYALI S.A.', 'ELECTRO NORTE S.A.', 'GASES DEL PACIFICO S.A.C.',
          'SEAL S.A.', 'ELECTRO CENTRO S.A.', 'ELECTRO SUR S.A.', 'ELECTRO DUNAS S.A.A.', 'ELECTRO SUR ESTE S.A.',
          'LUZ DEL SUR S.A.A.', 'ELECTRO ORIENTE S.A.', 'HIDRANDINA S.A.A.', 'CALIDDA - GAS NATURAL DE LIMA Y CALLAO S.R.L.', '',
          '', '', '', '', '',]

def clean_data(value):
    for i in range(1, 36, 2):
        table.children[i].value = 'Completar'

def expediente_data(value):
    sgu_data = data_sgu.loc[data_sgu['Expediente'] == IntText1.value]
    rur_data = data_rur.loc[data_rur['Expediente'] == IntText1.value]
    if len(sgu_data) != 0:
        data = sgu_data.values[0]
        table.children[5].value  = data[1].title() # Recurrente
        table.children[13].value = data[3].capitalize() # Materia
        table.children[17].value = str(data[4]) # Suministro
        table.children[21].value = data[5].title() # Direccion real
        table.children[25].value = data[6].title() # Direccion procesal
        date = str(data[8]).replace(' ', '').split('-')
        table.children[35].value = date[2][:2] + '/' + date[1] + '/' + date[0] # Fecha de ingreso
        if len(rur_data) != 0:
            data = rur_data.values[0]
            table.children[33].value  = data[6][:10] # Fecha de reclamo
            table.children[7].value   = data[7].replace('N° ', '') # Resolucion
            table.children[11].value  = data[8][:10] # Fecha de resolucion
        if data[2] in Dropdown1.options:
            Dropdown1.value = data[2]
        if 'APELACION' in data[7]:
            Dropdown2.value = 'RECURSO DE APELACION'
        elif 'QUEJA' in data[7]:
            Dropdown2.value = 'RECURSO DE QUEJA'
        elif 'CAUTELAR' in data[7]:
            Dropdown2.value = 'RECURSO DE MEDIDA CAUTELAR'
        else:
            Dropdown2.value = 'VERIFICACION DE CUMPLIMIENTO'
    elif len(rur_data) != 0:
        data = rur_data.values[0]
        table.children[5].value   = data[1].title() # Recurrente
        table.children[13].value  = data[3].capitalize() # Materia
        table.children[17].value  = str(data[4]) # Suministro
        table.children[35].value  = data[10][:10] # Fecha de ingreso
        table.children[33].value  = data[6][:10] # Fecha de reclamo
        table.children[7].value   = data[7].replace('N° ', '') # Resolucion
        table.children[11].value  = data[8][:10] # Fecha de resolucion
        if data[2] in concer:
            Dropdown1.value = ru2sgu[concer.index(data[2])]
        if 'APELACION' in data[9]:
            Dropdown2.value = 'RECURSO DE APELACION'
        elif 'QUEJA' in data[9]:
            Dropdown2.value = 'RECURSO DE QUEJA'
        elif 'CAUTELAR' in data[9]:
            Dropdown2.value = 'RECURSO DE MEDIDA CAUTELAR'
        else:
            Dropdown2.value = 'VERIFICACION DE CUMPLIMIENTO'
    else:
        table.children[5].value   = 'Completar' # Recurrente**
        table.children[9].value   = 'Completar' # Concesionaria**
        table.children[13].value  = 'Completar' # Materia**
        table.children[17].value  = 'Completar' # Suministro**
        table.children[21].value  = 'Completar' # Direccion real
        table.children[25].value  = 'Completar' # Direccion procesal
        table.children[35].value  = 'Completar' # Fecha de ingreso**
        table.children[33].value  = 'Completar' # Fecha de reclamo
        table.children[7].value   = 'Completar' # Resolucion
        table.children[11].value  = 'Completar' # Fecha de resolucion

def concesionaria(change):
    value = change.new
    table.children[9].value = nomruc[conces.index(value)]

def expediente(change):
    value = change.new
    Text2.value = str(value)

def haysap(change):
    try:
        if esfecha(table.children[33].value) and esfecha(table.children[11].value):
            dh = np.busday_count(num2dha(table.children[33].value), num2dha(table.children[11].value))
            if dh <= 30:
                Button5.button_style = 'success'
            elif dh > 30 and dh < 33:
                Button5.button_style = 'warning'
            elif dh >= 33:
                Button5.button_style = 'danger'
        else:
            Button5.button_style = 'info'
    except:
        Button5.button_style = 'info'

def esexte(change):
    try:
        if esfecha(table.children[19].value) and esfecha(table.children[27].value):
            dh = np.busday_count(num2dha(table.children[19].value), num2dha(table.children[27].value))
            if dh <= 15:
                Button6.button_style = 'success'
            elif dh > 15 and dh < 18:
                Button6.button_style = 'warning'
            elif dh >= 18:
                Button6.button_style = 'danger'
        else:
            Button6.button_style = 'info'
    except:
        Button6.button_style = 'info'

def esfecha(fecha):
    valid = False
    if '-' in fecha or '/' in fecha:
        if len(fecha.replace(' ', '').split('/')) == 3:
            date = fecha.replace(' ', '').split('/')
            if len(date[0]) == 2 and len(date[1]) == 2 and len(date[2]) == 4:
                valid = True
        elif len(fecha.replace(' ', '').split('-')) == 3:
            date = fecha.replace(' ', '').split('-')
            if len(date[0]) == 2 and len(date[1]) == 2 and len(date[2]) == 4:
                valid = True
    return valid

def num2dha(fecha):
    if '/' in fecha and len(fecha.split("/")) == 3:
        fecha = fecha.replace(' ', '').split("/")
        fecha = fecha[2] + '-' + fecha[1] + '-' + fecha[0]
    elif '-' in fecha and len(fecha.split("-")) == 3:
        fecha = fecha.replace(' ', '').split("-")
        fecha = fecha[2] + '-' + fecha[1] + '-' + fecha[0]
    return fecha

def numero2mes(fecha):
    if esfecha(fecha):
        meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
        fecha = fecha.replace(' ', '').replace('-', '/').split('/')
        fecha = str(fecha[0]) + ' de ' + meses[int(fecha[1].replace('0', '')) - 1].lower() + ' de ' + fecha[2]
    else:
        fecha = '[[ ]]'
    return fecha

def mes2numero(fecha):
    dia   = '00'
    mes   = '00'
    año   = '0000'
    if fecha.replace(' ', '') != '':
        dates = fecha.split(' ');
        fecha = []
        for i in dates:
            if i != '':
                fecha.append(i)
        if len(fecha) == 5:
            dia   = fecha[0].replace('o', '0').replace('g', '9').replace(' ', '')
            mes   = fecha[2].replace('setiembre', 'Septiembre').replace('Setiembre', 'Septiembre')
            año   = fecha[4].replace('o', '0').replace('g', '9').replace(' ', '')
            fecha = [dia, fecha[1], mes, fecha[3], año]
            fecha = ' '.join(fecha)
            meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
            digit = ''
            for i in fecha:
                if i.isdigit():
                    digit = digit + i
            if len(digit) == 5 or len(digit) == 6:
                if len(digit) == 5:
                    dia = '0' + digit[0]
                else:
                    dia = digit[:2]
                for j in range(len(meses)):
                    if meses[j] in fecha or meses[j].lower() in fecha:
                        mes = str(j + 1)
                        if len(mes) == 1:
                            mes = '0' + mes
                año = digit[-4:]
    fecha = dia + '/' + mes + '/' + año
    if fecha == '00/00/0000':
        fecha = 'Completar'
    return fecha

def calidda(pages):
    kword01 = ['Osinergmin presentado el']               #FECHA DE RECLAMO INICIAL
    kword02 = [' por']
    kword03 = ['contra la Resolución N']                 #RESOLUCION IMPUGNADA
    kword04 = [' emitida', ' ']
    kword05 = ['emitida con fecha']                      #FECHA DE RESOLUCION
    kword06 = [', que resolvió', ',']
    kword07 = ['instancia declarando']                   #RESULTADO DE LA RESOLUCION
    kword08 = ['el reclamo', ' ']
    kword09 = ['informamos que el']                      #FECHA DE APELACION *
    kword10 = [', el', ',']
    kword11 = ['REC']                                    #DOC QUE SE ELEVA A JARU
    kword12 = [' ']
    kword13 = ['Lima,']                                  #FECHA DEL DOC QUE SE ELEVA A JARU
    kword14 = ['Señores:']
    kwordss = [kword01, kword03, kword05, kword07, kword09, kword11, kword13]
    kwordse = [kword02, kword04, kword06, kword08, kword10, kword12, kword14]
    answer  = ['Completar'] * len(kwordss)
    foundw  = [0]  * len(kwordse)
    IntProgress1.max = len(pages)
    for page in pages:
        text = pytesseract.image_to_string(page, lang = 'spa')
        text = text.replace('?', ' ').replace('\n', ' ').split()
        text = " ".join(text)
        for kword in range(len(kwordss)):
            if foundw[kword] == 0:
                for i in kwordss[kword]:
                    resultado1 = re.search(i, text)
                    if resultado1 != None:
                        inicio = resultado1.end() + 1
                        for j in kwordse[kword]:
                            resultado2 = re.search(j, text[inicio:])
                            if resultado2 != None:
                                final = resultado2.start() + inicio
                                word  = text[inicio:final]
                                word  = re.sub('\n', ' ', word)
                                word  = word.replace("| ", "")
                                answer[kword] = word
                                foundw[kword] = 1
                                break
                        break
        IntProgress1.value = IntProgress1.value + 1
        if sum(foundw) == len(kwordse):
            IntProgress1.value = len(pages)
            break
    table.children[7].value  = answer[1].replace(' ', '') # Resolucion
    table.children[11].value = mes2numero(answer[2]) # Fecha de resolucion
    table.children[15].value = answer[3].lower().replace(' ', '') # Resultado
    table.children[27].value = mes2numero(answer[4]) # Fecha de apelacion
    table.children[31].value = 'REC2' + answer[5].replace(' ', '') # Elevacion
    table.children[33].value = mes2numero(answer[0]) # Fecha de reclamo
    table.children[35].value = mes2numero(answer[6]) # Fecha de elevacion

def enel(pages):
    IntProgress1.max = 2
    IntProgress1.value = 1
    full = ''
    for page in pages:
        text = pytesseract.image_to_string(page, lang = 'spa')
        text = text.replace('?', ' ').replace('\n', ' ').split()
        text = " ".join(text) + ' newpage '
        full = full + text

    i = re.search('newpage', full).end() + 1
    if re.search('RESOLUCIÓN N', full[i:]) != None:
        j = re.search('RESOLUCIÓN N', full[i:]).end() + 1
        k = re.search(' de', full[i + j:].lower()).start() + i + j
        resol = full[i + j:k]
    elif re.search('Resolución N', full[i:]) != None:
        j = re.search('Resolución N', full[i:]).end() + 1
        k = re.search(' de', full[i + j:].lower()).start() + i + j
        resol = full[i + j:k]
    else:
        resol = 'Completar'

    full = full.split(' ')
    fechas = []
    for i in full:
        n = 0
        for j in i:
            if j.isdigit():
                n = n + 1
        if n == 8 and i[-4:-2] == '20':
            i = i.replace('-', '').replace('/', '').replace('.', '').replace(' ', '')
            fechas.append(i[:2] + '/' + i[2:4] + '/' + i[-4:])
        elif n == 7 and i[-4:-2] == '20':
            i = i.replace('-', '').replace('/', '').replace('.', '').replace(' ', '')
            fechas.append('0' + i[0] + '/' + i[1:3] + '/' + i[-4:])

    docum = 'Completar'
    for i in full:
        n = 0
        for j in i:
            if j.isdigit():
                n = n + 1
        if n >= 9:
            docum = i
            break

    resul = 'Completar'
    for i in range(len(full)):
        if full[i] == 'x' or full[i] == 'X' or full[i] == 'x|' or full[i] == 'X|':
            resul = full[i + 1] + full[i + 2] + full[i + 3]
            if 'INFUNDADO' in resul:
                resul = 'infundado'
                break
            elif 'IMPROCEDENTE' in resul:
                resul = 'improcedente'
                break
            elif 'PENDIENTE' in resul:
                resul = 'pendiente de pronunciamiento'
                break
            elif 'FUNDADO EN' in resul:
                resul = 'fundado en parte'
                break
            elif 'FUNDADO' in resul:
                resul = 'fundado'
                break
            else:
                resul = 'Completar'

    if len(fechas) > 1:
        if datetime.strptime(fechas[0], '%d/%m/%Y') > datetime.strptime(fechas[1], '%d/%m/%Y'):
            table.children[33].value = fechas[1]
            if len(fechas) > 2:
                table.children[11].value = fechas[2]
            else:
                table.children[11].value = 'Completar'
            if len(fechas) > 3:
                if datetime.strptime(fechas[3], '%d/%m/%Y') > datetime.strptime(fechas[2], '%d/%m/%Y'):
                    table.children[23].value = fechas[3]
            else:
                table.children[23].value = 'Completar'
            table.children[35].value = fechas[0]
        else:
            table.children[33].value = fechas[0]
            table.children[11].value = fechas[1]
            if len(fechas) > 2:
                if datetime.strptime(fechas[2], '%d/%m/%Y') > datetime.strptime(fechas[1], '%d/%m/%Y'):
                    if len(fechas) > 3:
                        table.children[23].value = fechas[3]
                    else:
                        table.children[23].value = 'Completar'
            fechas_datetime = [datetime.strptime(fecha, '%d/%m/%Y') for fecha in fechas]
            table.children[35].value = max(fechas_datetime).strftime('%d/%m/%Y')
        table.children[27].value = fechas[-1]
    elif len(fechas) > 0:
        table.children[35].value = fechas[0]
        table.children[33].value = 'Completar'
        table.children[11].value = 'Completar'
        table.children[23].value = 'Completar'
        table.children[27].value = 'Completar'
    else:
        table.children[35].value = 'Completar'
        table.children[33].value = 'Completar'
        table.children[11].value = 'Completar'
        table.children[23].value = 'Completar'
        table.children[27].value = 'Completar'
    table.children[7].value  = resol # Resolucion
    table.children[15].value = resul # Resultado
    table.children[31].value = docum.replace(' ', '') # Elevacion
    IntProgress1.value = 2

def luzdelsur(pages):
    IntProgress1.max = 2
    IntProgress1.value = 1
    text   = pytesseract.image_to_string(pages[-1], lang = 'spa')
    text   = text.replace('?', ' ').replace('\n', ' ').split()
    text   = " ".join(text)
    full   = text.split(' ')

    meses  = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 
              'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    numer  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    fechas = []
    fecha  = ''
    for i in range(len(full)):
        n = 0
        for j in full[i]:
            if j.isdigit():
                n = n + 1
        if (n == 8 or n == 7) and full[i].replace(',', '')[-4:-2] == '20':
            if n == 7:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append('0' + n[0] + '/' + n[1:3] + '/' + n[-4:])
            else:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append(n[:2] + '/' + n[2:4] + '/' + n[-4:])
        for k in range(len(meses)):
            if meses[k] == full[i] or meses[k].upper() == full[i]:
                if len(full[i - 2]) == 1:
                    fecha = '0' + full[i - 2] + '/' + numer[k] + '/' + full[i + 2]
                else:
                    fecha = full[i - 2] + '/' + numer[k] + '/' + full[i + 2]
                fechas.append(fecha.replace(',', '').replace('.', ''))

    names = []
    for i in full:
        if 'DAR' in i or 'DRR' in i:
            if i[-1] == '.':
                names.append(i[:-1])
            else:
                names.append(i)

    res  = 'Completar'
    for i in range(len(full)):
        if full[i] == 'x' or full[i] == 'X' or full[i] == 'x|' or full[i] == 'X|':
            res = full[i + 1] + full[i + 2] + full[i + 3]
            if 'INFUNDADO' in res:
                res = 'infundado'
                break
            elif 'IMPROCEDENTE' in res:
                res = 'improcedente'
                break
            elif 'FUNDADO EN' in res:
                res = 'fundado en parte'
                break
            elif 'FUNDADO' in res:
                res = 'fundado'
                break

    if len(names) > 1:
        table.children[31].value = names[0].replace(' ', '') # Elevacion
        table.children[7].value  = names[1].replace(' ', '') # Resolucion
    elif len(names) > 0:
        table.children[31].value = names[0].replace(' ', '') # Elevacion
    else:
        table.children[31].value = 'Completar'
        table.children[7].value  = 'Completar'
    table.children[15].value = res # Resultado
    if len(fechas) > 3:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[23].value = fechas[2] # Fecha de reconsideracion
        table.children[27].value = fechas[3] # Fecha de apelacion
    elif len(fechas) > 2:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[27].value = fechas[2] # Fecha de apelacion
        table.children[23].value = 'Completar'
    elif len(fechas) > 1:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[27].value = 'Completar' # Fecha de apelacion
        table.children[23].value = 'Completar'
    elif len(fechas) > 0:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[27].value = 'Completar' # Fecha de apelacion
    else:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[27].value = 'Completar' # Fecha de apelacion
    IntProgress1.value = 2

def hidrandina(pages):
    IntProgress1.max = 2
    IntProgress1.value = 1
    texto = ''
    for page in pages:
        text = pytesseract.image_to_string(page, lang = 'spa')
        text = text.replace('?', ' ').replace('\n', ' ').split()
        text = " ".join(text) + ' newpage '
        texto = texto + text
    full  = texto.split(' ')

    meses  = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 
             'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    numer  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    fechas = []
    fecha  = ''
    for i in range(len(full)):
        n = 0
        for j in full[i]:
            if j.isdigit():
                n = n + 1
        if (n == 8 or n == 7) and full[i].replace(',', '')[-4:-2] == '20' and 'GC' not in full[i]:
            if n == 7:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append('0' + n[0] + '/' + n[1:3] + '/' + n[-4:])
            else:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append(n[:2] + '/' + n[2:4] + '/' + n[-4:])
        for k in range(len(meses)):
            if meses[k] == full[i].lower():
                fecha = full[i - 2] + '/' + numer[k] + '/' + full[i + 2].replace(',', '').replace('.', '')
                n = 0
                for j in fecha:
                    if j.isdigit():
                        n = n + 1
                if len(fecha) == 10 and n == 8:
                    fechas.append(fecha.replace(',', '').replace('.', ''))

    names = []
    for i in full:
        n = 0
        for j in i:
            if j.isdigit():
                n = n + 1
        if (('GC-' in i or 'GC/' in i) and len(i) >= 11) or (n == 11 and len(i) == 11):
            names.append(i)

    res = 'Completar'
    for i in range(len(full)):
        if full[i] == 'declarando':
            res = full[i + 1]
            break
    
    if len(names) > 3:
        table.children[31].value = names[0].replace(' ', '') # Elevacion
        table.children[7].value  = names[3].replace(' ', '') # Resolucion
    elif len(names) > 0:
        table.children[31].value = names[0].replace(' ', '') # Elevacion
    else:
        table.children[31].value = 'Completar'
        table.children[7].value  = 'Completar'
    table.children[15].value = res.lower() # Resultado
    if len(fechas) > 7:
        table.children[33].value = fechas[5] # Fecha de reclamo
        table.children[11].value = fechas[6] # Fecha de resolucion
        table.children[23].value = fechas[7] # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = fechas[1] # Fecha de apelacion
    elif len(fechas) > 6:
        table.children[33].value = fechas[5] # Fecha de reclamo
        table.children[11].value = fechas[6] # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = fechas[1] # Fecha de apelacion
    elif len(fechas) > 5:
        table.children[33].value = fechas[5] # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = fechas[1] # Fecha de apelacion
    elif len(fechas) > 1:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = fechas[1] # Fecha de apelacion
    elif len(fechas) > 0:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    else:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = 'Completar' # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    IntProgress1.value = 2
    
def electronoroeste(pages):
    IntProgress1.max = 2
    IntProgress1.value = 1
    texto = ''
    for page in [0, -1]:
        text = pytesseract.image_to_string(pages[page], lang = 'spa')
        text = text.replace('?', ' ').replace('\n', ' ').split()
        text = " ".join(text) + ' newpage '
        texto = texto + text
    full  = texto.split(' ')
    
    meses  = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 
              'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    numer  = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    fechas = []
    fecha  = ''
    for i in range(len(full)):
        n = 0
        for j in full[i]:
            if j.isdigit():
                n = n + 1
        if (n == 8 or n == 7) and full[i].replace(',', '')[-4:-2] == '20' and len(full[i]) <= 12:
            if n == 7:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append('0' + n[0] + '/' + n[1:3] + '/' + n[-4:])
            else:
                n = full[i].replace('-', '').replace('/', '').replace('.', '').replace(',', '')
                fechas.append(n[:2] + '/' + n[2:4] + '/' + n[-4:])
        for k in range(len(meses)):
            if meses[k] == full[i] or meses[k].upper() == full[i]:
                if len(full[i - 2]) == 1:
                    fecha = '0' + full[i - 2] + '/' + numer[k] + '/' + full[i + 2]
                else:
                    fecha = full[i - 2] + '/' + numer[k] + '/' + full[i + 2]
                fechas.append(fecha.replace(',', '').replace('.', ''))
    fechas = list(set(fechas))
    fechas_datetime = [datetime.strptime(fecha, '%d/%m/%Y') for fecha in fechas]
    fechas_datetime = sorted(fechas_datetime)
    fechas = [fecha.strftime('%d/%m/%Y') for fecha in fechas_datetime]

    elev = []
    for i in full:
        if 'ENOSA-' in i:
            elev.append(i)

    reso = []
    for i in full:
        n = 0
        for j in i:
            if j.isdigit():
                n = n + 1
        if (n == 11 or n == 10 or n == 9) and i.replace(',', '').replace('.', '').isdigit():
            reso.append(i.replace(',', '').replace('.', ''))
    reso = list(set(reso))

    res  = 'Completar'
    for i in range(len(full)):
        if full[i] == 'x' or full[i] == 'X' or full[i] == 'x|' or full[i] == 'X|':
            res = full[i + 1] + full[i + 2] + full[i + 3]
            if 'INFUNDADO' in res:
                res = 'infundado'
                break
            elif 'IMPROCEDENTE' in res:
                res = 'improcedente'
                break
            elif 'FUNDADO EN' in res:
                res = 'fundado en parte'
                break
            elif 'FUNDADO' in res:
                res = 'fundado'
                break    
    
    if len(elev) > 0:
        table.children[31].value = elev[0].replace(' ', '') # Elevacion
    else:
        table.children[31].value = 'Completar' # Elevacion
    if len(reso) > 0:
        table.children[7].value  = reso[0].replace(' ', '') # Resolucion
    else:
        table.children[7].value  = 'Completar' # Resolucion
    table.children[15].value = res.lower() # Resultado
    if len(fechas) > 4:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[23].value = fechas[2] # Fecha de reconsideracion
        table.children[35].value = fechas[-1] # Fecha de elevacion
        table.children[27].value = fechas[3] # Fecha de apelacion
    elif len(fechas) > 3:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[-1] # Fecha de elevacion
        table.children[27].value = fechas[2] # Fecha de apelacion
    elif len(fechas) > 2:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = fechas[1] # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[-1] # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    elif len(fechas) > 1:
        table.children[33].value = fechas[0] # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[-1] # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    elif len(fechas) > 0:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = fechas[0] # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    else:
        table.children[33].value = 'Completar' # Fecha de reclamo
        table.children[11].value = 'Completar' # Fecha de resolucion
        table.children[23].value = 'Completar' # Fecha de reconsideracion
        table.children[35].value = 'Completar' # Fecha de elevacion
        table.children[27].value = 'Completar' # Fecha de apelacion
    IntProgress1.value = 2
    
def analisis_concesionaria(pages):
    if Dropdown1.value == 'Enel Distribucion Peru SAA':
        enel(pages)
    elif Dropdown1.value == 'CALIDDA - GAS NATURAL DE LIMA Y CALLAO S.R.L.':
        calidda(pages)
    elif Dropdown1.value == 'LUZ DEL SUR S.A.A.':
        luzdelsur(pages)
    elif Dropdown1.value == 'HIDRANDINA S.A.A.':
        hidrandina(pages)
    elif Dropdown1.value == 'ELECTRO NOROESTE S.A.':
        electronoroeste(pages)

def analisis_local(value):
    IntProgress1.value = 0
    pages = []
    for i in FileUpload1.data:
        pages = pages + convert_from_bytes(i, 150)
    analisis_concesionaria(pages)

def analisis_online(value):
    IntProgress1.value = 0
    pdfurl = requests.get(Text1.value)
    pages  = convert_from_bytes(pdfurl.content, 150)
    IntProgress1.max = len(pages)
    analisis_concesionaria(pages)

def create_word(value):
    for i in range(1, 36, 2):
        if table.children[i].value == '' or table.children[i].value == 'Completar' or table.children[i].value == 'completar':
              table.children[i].value = '[[ ]]'
    if Checkbox1.value:
        if Dropdown4.value == 'Física':
            doc = DocxTemplate("Modelo_NP2.docx")
        elif Dropdown4.value == 'Física notarial':
            doc = DocxTemplate("Modelo_NN2.docx")
        else:
            doc = DocxTemplate("Modelo_NE2.docx")
    else:
        if Dropdown4.value == 'Física':
            doc = DocxTemplate("Modelo_NP1.docx")
        elif Dropdown4.value == 'Física notarial':
            doc = DocxTemplate("Modelo_NN1.docx")
        else:
            doc = DocxTemplate("Modelo_NE1.docx")
    if Dropdown3.value == 'Masculino':
        articulo = 'el'
        genero = 'Sr'
        tipo = 'propio'
    else:
        articulo = 'la'
        genero = 'Sra'
        tipo = 'propia'
    context = {
        "expediente": str(IntText1.value),
        "reclamante": table.children[1].value,
        "recurrente": table.children[5].value,
        "concesionaria": table.children[9].value,
        "materia": table.children[13].value,
        "suministro": str(table.children[17].value).replace(' ', ''),
        "domicilioreal": table.children[21].value,
        "domicilioprocesal": table.children[25].value,
        "correo": table.children[29].value.replace(' ', ''),
        "resolucion": str(table.children[7].value),
        "fechaapelacion": numero2mes(table.children[27].value),
        "articulo": articulo,
        "genero": genero,
        "tipo": tipo,
        "fechareclamo": numero2mes(table.children[33].value),
        "motivo": table.children[3].value,
        "fecharesolucion": numero2mes(table.children[11].value),
        "resultado": table.children[15].value.replace(' ', ''),
        "fechanotificacion": numero2mes(table.children[19].value),
        "fechareconsideracion": numero2mes(table.children[23].value),
        "elevacion": str(table.children[31].value),
        "fechaelevacion": numero2mes(table.children[35].value)
    }
    doc.render(context)
    doc.save('Replace.docx')

    doc = Document('Replace.docx')
    docx_blocks(doc, tercero = Checkbox2.value)
    docx_blocks(doc, reconsideracion = Checkbox3.value)
    docx_blocks(doc, oficio = Checkbox4.value)
    docx_blocks(doc, actadeatencion = Checkbox5.value)
    docx_blocks(doc, solicitud = Checkbox6.value)
    docx_blocks(doc, complementaria = Checkbox7.value)
    docx_blocks(doc, jaru = Checkbox8.value)
    docx_blocks(doc, acumulacion = Checkbox9.value)
    doc.save(Text2.value + '.docx')
    
    gfile = drive.CreateFile({'parents': [{'id': '1kGuruJPaDCIF2jngmrDyMs-4XqWkCGR4'}]})
    gfile.SetContentFile(Text2.value + '.docx')
    gfile.Upload()
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format('1kGuruJPaDCIF2jngmrDyMs-4XqWkCGR4')}).GetList()
    docname   = []
    docid     = []
    for file in file_list:
        docname.append(file['title'])
        docid.append(file['id'])
    if Text2.value + '.docx' in docname:
        htmlid = docid[docname.index(Text2.value + '.docx')]
        width  = "590px"
        height = "580px"
        border = "0"
        urls   = "https://drive.google.com/file/d/" + htmlid + "/preview"
        HTML2.value = f"<iframe src={urls} width={width} height={height} frameborder={border}>"
    else:
        htmlid = "1evMTdiqhMw6Mp6SWZy-zeR6HVaJs-yFr"
        width  = "590px"
        height = "580px"
        border = "0"
        urls   = "https://drive.google.com/file/d/" + htmlid + "/preview"
        HTML2.value = f"<iframe src={urls} width={width} height={height} frameborder={border}>"


In [6]:
datos = ['Reclamante', 'Recurrente', 'Concesionaria', 'Materia', 'Suministro', 'Dirección real', 'Dirección procesal',
         'Correo electrónico', 'Fecha de reclamo', 'Motivo', 'Resolución', 'Fecha de resolución', 'Resultado',
         'Fecha de notificación', 'Fecha de reconsideración', 'Fecha de apelación', 'Elevación', 'Fecha de elevación']
conce = ['ELECTRO DUNAS S.A.A.', 'LUZ DEL SUR S.A.A.', 'ELECTRO UCAYALI S.A.', '*ELECTRO PUNO S.A.A.',
         'ADINELSA', 'HIDRANDINA S.A.A.', 'SEAL S.A.', 'CONTUGAS S.A.C', 'EMSEMSA', 'ELECTRO NORTE S.A.',
         'ELECTRO SUR S.A.', 'GASES DEL PACIFICO S.A.C.', 'ELECTRO NOROESTE S.A.', 'ELECTRO ORIENTE S.A.',
         'ELECTRO CENTRO S.A.', 'EILHICHA S.A.', 'Enel Distribucion Peru SAA', 'COELVISA',
         'CALIDDA - GAS NATURAL DE LIMA Y CALLAO S.R.L.', 'ELECTRO SUR ESTE S.A.']
recurs = ['RECURSO DE APELACION', 'RECURSO DE QUEJA', 'RECURSO DE MEDIDA CAUTELAR',
          'VERIFICACION DE CUMPLIMIENTO']
table = ipw.GridBox(layout = ipw.Layout(width = "100%", grid_template_columns = "15% 35% 15% 35%", grid_template_rows = "repeat(9, auto)"))
filas = list(range(9, 0, -1))
for i in range(9):
    label1 = ipw.Label(datos[i])
    label3 = ipw.Label(datos[i * 2 + filas[i]])
    textarea2 = ipw.Textarea(layout = ipw.Layout(width = "90%"))
    textarea4 = ipw.Textarea(layout = ipw.Layout(width = "90%"))
    table.children += (label1, textarea2, label3, textarea4)
table.children[33].observe(haysap, names = 'value')
table.children[11].observe(haysap, names = 'value')
table.children[19].observe(esexte, names = 'value')
table.children[27].observe(esexte, names = 'value')
Label1       = ipw.Label(value = 'Número de expediente', layout = ipw.Layout(width = "14.5%", margin = "0px 2% 0px 0px"))
Label2       = ipw.Label(value = 'Nombre del documento', layout = ipw.Layout(width = "14.5%", margin = "0px 1.15% 0px 0px"))
IntText1     = ipw.IntText(layout = ipw.Layout(width = "15%", margin = "0px 2% 0px 0px"))
IntText1.observe(expediente, names = 'value')
Text1        = ipw.Text(layout = ipw.Layout(width = "50%", margin = "0px 2% 0px 0px"))
Text2        = ipw.Text(layout = ipw.Layout(width = "32.65%", margin = "0px 3.7% 0px 0px"))
Dropdown1    = ipw.Dropdown(options = conce, layout = ipw.Layout(width = "29%", margin = "0px 2% 0px 0px"))
Dropdown1.observe(concesionaria, names = 'value')
Dropdown2    = ipw.Dropdown(options = recurs, layout = ipw.Layout(width = "19%", margin = "0px 2% 0px 0px"))
Dropdown3    = ipw.Dropdown(options = ['Masculino', 'Femenino'], layout = ipw.Layout(width = "13.8%", margin = "0px 0.5% 0px 0px"))
Dropdown4    = ipw.Dropdown(options = ['Física', 'Física notarial', 'Electrónica'], layout = ipw.Layout(width = "13.8%", margin = "0px 1% 0px 0px"))
FileUpload1  = ipw.FileUpload(description = 'PDFs', accept = ".pdf", multiple = True, layout = ipw.Layout(width = "13%", margin = "0px 3.3% 0px 0px"))
Button1      = ipw.Button(description = 'Extraer información', button_style = 'success', layout = ipw.Layout(width = "15%", margin = "0px 0px 0px 0px"))
Button1.on_click(expediente_data)
Button2      = ipw.Button(description = 'Analizar expediente', button_style = 'success', layout = ipw.Layout(width = "15%", margin = "0px 2% 0px 0px"))
Button2.on_click(analisis_local)
Button3      = ipw.Button(description = 'Analizar expediente', button_style = 'success', layout = ipw.Layout(width = "15%", margin = "0px 0px 0px 0px"))
Button3.on_click(analisis_online)
Button4      = ipw.Button(description = 'Generar resolución', button_style = 'success', layout = ipw.Layout(width = "48%", margin = "0px 0px 0px 0px"))
Button4.on_click(create_word)
Button5      = ipw.Button(description = 'SAP', button_style = 'success', layout = ipw.Layout(width = "5%", margin = "0px 1% 0px 0px"))
Button6      = ipw.Button(description = 'EXT', button_style = 'success', layout = ipw.Layout(width = "5%", margin = "0px 1% 0px 0px"))
Button7      = ipw.Button(description = 'Limpiar datos', button_style = 'success', layout = ipw.Layout(width = "19.5%", margin = "0px 0px 0px 0px"))
Button7.on_click(clean_data)
Checkbox1    = ipw.Checkbox(value = False, description = 'Incluye reclamante', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox2    = ipw.Checkbox(value = False, description = 'Incluye tercero', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox3    = ipw.Checkbox(value = False, description = 'Incluye reconsideración', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox4    = ipw.Checkbox(value = False, description = 'Incluye oficio', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox5    = ipw.Checkbox(value = False, description = 'Incluye acta de atención', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox6    = ipw.Checkbox(value = False, description = 'Incluye solicitud de información', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
Checkbox7    = ipw.Checkbox(value = False, description = 'Incluye información complementaria', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 2% 0px 0px', border = '0px'))
Checkbox8    = ipw.Checkbox(value = False, description = 'Incluye resolución JARU', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 1% 0px 0px', border = '0px'))
Checkbox9    = ipw.Checkbox(value = False, description = 'Incluye acumulación', indent = False, layout = ipw.Layout(width = '20%', margin = '0px 0px 0px 0px', border = '0px'))
IntProgress1 = ipw.IntProgress(value = 100, min = 0, bar_style = 'success', orientation = 'horizontal', layout = ipw.Layout(width = "96.5%", margin = "0px 0px 0px 0px"))
width  = "595px"
height = "580px"
border = "0"
urls   = "https:&#x2F;&#x2F;www.canva.com&#x2F;design&#x2F;DAFeofDDZ3A&#x2F;view?embed"
HTML1  = ipw.HTML(value = f"<iframe src={urls} width={width} height={height} frameborder={border}>", layout = ipw.Layout(margin = "0px 0px 0px 0px"))
width  = "590px"
height = "580px"
border = "0"
urls   = "https://drive.google.com/file/d/1evMTdiqhMw6Mp6SWZy-zeR6HVaJs-yFr/preview"
HTML2  = ipw.HTML(value = f"<iframe src={urls} width={width} height={height} frameborder={border}>", layout = ipw.Layout(margin = "0px 0px 0px 47px"))

HBox1 = ipw.HBox([Label1, IntText1, Dropdown1, Dropdown2, Button1], layout = ipw.Layout(justify_content = 'flex-start', width = "96.5%", margin = '0px 0px 0px 0px'))
HBox2 = ipw.HBox([FileUpload1, Button2, Text1, Button3], layout = ipw.Layout(justify_content = 'flex-start', width = "96.5%", margin = '1% 0px 0.5% 0px'))
HBox3 = ipw.HBox([Checkbox1, Checkbox2, Checkbox3, Checkbox4, Checkbox5, Checkbox6], layout = ipw.Layout(justify_content = 'flex-start', width = "96.5%", margin = '0.5% 0px 0.5% 0px'))
HBox4 = ipw.HBox([Checkbox7, Checkbox8, Checkbox9, Dropdown3, Dropdown4, Button5, Button6, Button7], layout = ipw.Layout(justify_content = 'flex-start', width = "96.5%", margin = '0px 0px 0px 0px'))
HBox5 = ipw.HBox([Label2, Text2, Button4], layout = ipw.Layout(justify_content = 'flex-start', width = "96.5%", margin = '1% 0px 2% 0px'))
HBox6 = ipw.HBox([HTML1, HTML2], layout = ipw.Layout(justify_content = 'flex-start', margin = '2% 0px 0px 0px'))
VBox5 = ipw.VBox([HBox1, HBox2, IntProgress1, HBox3, HBox4, HBox5, table, HBox6], layout = ipw.Layout(margin = '0px 0px 0px 2.25%'))
display(VBox5)